# Khai báo thư viện

In [1]:
import numpy as np

# Đọc dữ liệu

In [2]:
data = open('dinos.txt', 'r').read()
data = data.lower()

chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

print('Có tổng %d ký tự và %d ký tự riêng biệt' % (data_size, vocab_size))

Có tổng 19909 ký tự và 27 ký tự riêng biệt


# Xử dữ liệu

### Tạo dictionary hàm chuyển từ từ sang mã số, và ngược lại

In [3]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }

print(char_to_ix)
print(ix_to_char)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### Tạo dữ liệu để thực hiện training

In [4]:
examples = data.splitlines()
examples = [x.lower().strip() for x in examples]
examples[:10]

np.random.seed(0)
np.random.shuffle(examples)

# Xây dựng các hàm để xử lý

In [5]:
def softmax(x):
    e_x = np.exp((x - np.max(x)))
    return e_x / e_x.sum(axis=0)

In [6]:
def name_from_indices(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]
    return txt

# Xây dựng model

In [7]:
class Simple_RNN():
    def __init__(self, vocab_size, n_a = 64):
        self.vocab_size = vocab_size
        self.n_a = n_a
        self.a_prev, self.parameters = self.initialize_parameters()
    
    def initialize_parameters(self):
        np.random.seed(0)
        a_prev = np.zeros((self.n_a, 1))
        
        np.random.seed(1)
        Wax = np.random.randn(self.n_a, self.vocab_size)*0.01 # đầu vào tới ẩn
        Waa = np.random.randn(self.n_a, self.n_a)*0.01 # ẩn tới ẩn
        Wya = np.random.randn(self.vocab_size, self.n_a)*0.01 # ẩn tới đầu ra

        ba = np.zeros((self.n_a, 1)) # sai số ẩn
        by = np.zeros((self.vocab_size, 1)) # sai số đầu ra

        parameters = {'Wax': Wax, 'Waa': Waa, 'Wya': Wya, 'ba': ba,'by': by}
        return a_prev, parameters
    
    def train(self, X, Y):
        a_prev, parameters, loss = self.optimize(X, Y)
        
        self.a_prev = a_prev
        self.parameters = parameters
            
        return loss

    def sample(self, seed):
        Waa = self.parameters['Waa']
        Wax = self.parameters['Wax']
        Wya = self.parameters['Wya']
        by = self.parameters['by']
        ba = self.parameters['ba']

        a_prev = self.a_prev
        x = np.zeros((vocab_size, 1))
        indices = []
        
        for i in range(0, 50):
            at = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
            yt = softmax(np.dot(Wya, at) + by)
            a_prev = at

            np.random.seed(i + seed) 
            idx = np.random.choice(list(range(vocab_size)), p=yt.ravel())
            indices.append(idx)

            x = np.zeros((vocab_size, 1))
            x[idx] = 1
            seed += 1

            if idx == char_to_ix['\n']: break
        
        return indices

    def optimize(self, X, Y, learning_rate=0.01):        
        loss, cache = self.rnn_forward(X, Y, self.a_prev, self.parameters)
        
        gradients, a_prev = self.rnn_backward(X, Y, cache, self.parameters)
        gradients = self.clip(gradients, 5)

        parameters = self.update_parameters(gradients, self.parameters, learning_rate)
        return a_prev, parameters, loss

    def clip(self, gradients, maxValue):
        for gradient in gradients:
            np.clip(gradients[gradient], -maxValue, maxValue, out=gradients[gradient])
        
        return gradients

    def update_parameters(self, gradients, parameters, lr):

        parameters['Wax'] += -lr * gradients['dWax']
        parameters['Waa'] += -lr * gradients['dWaa']
        parameters['Wya'] += -lr * gradients['dWya']
        parameters['ba']  += -lr * gradients['dba']
        parameters['by']  += -lr * gradients['dby']

        return parameters

    def rnn_step_forward(self, a_prev, x, parameters):
        
        Wax = parameters['Wax']
        Waa = parameters['Waa']
        Wya = parameters['Wya']
        ba = parameters['ba']
        by = parameters['by']
        
        try:
            a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba) 
            yt_pred = softmax(np.dot(Wya, a_next) + by) 
        except:
            print(a_prev.shape, x.shape, parameters['Wax'].shape, parameters['Waa'].shape, parameters['Wya'].shape, parameters['ba'].shape, parameters['by'].shape)
        
        return a_next, yt_pred

    def rnn_step_backward(self, dy, gradients, x, a, a_prev, parameters):
        
        gradients['dWya'] += np.dot(dy, a.T)
        gradients['dby'] += dy

        # backprop vào h
        da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] 
        
        # backprop qua hàm tanh
        dtanh = (1 - a * a) * da 
        
        gradients['dba'] += dtanh
        gradients['dWax'] += np.dot(dtanh, x.T)
        gradients['dWaa'] += np.dot(dtanh, a_prev.T)
        gradients['da_next'] = np.dot(parameters['Waa'].T, dtanh)

        return gradients

    def rnn_forward(self, X, Y, a_prev, parameters):
        
        x, a, y_pred = {}, {}, {}
        a[-1] = np.copy(a_prev)
        loss = 0
        
        for t in range(len(X)):
            
            x[t] = np.zeros((self.vocab_size, 1))
            if (X[t] != None):
                x[t][X[t]] = 1
            
            a[t], y_pred[t] = self.rnn_step_forward(a[t-1], x[t], parameters)
            loss -= np.log(y_pred[t][Y[t], 0])
        
        cache = (y_pred, a, x)

        return loss, cache

    def rnn_backward(self, X, Y, cache, parameters):
        # Khởi tạo gradients thành một từ điển trống
        gradients = {}
        
        # Truy xuất từ cache và parameters
        (y_pred, a, x) = cache
        Waa = parameters['Waa']
        Wax = parameters['Wax']
        Wya = parameters['Wya']
        by = parameters['by']
        ba = parameters['ba']
        
        # mỗi cái phải được khởi tạo thành các vector zeros cùng kích thước với tham số tương ứng của nó
        gradients['dWax'] = np.zeros_like(Wax)
        gradients['dWaa'] = np.zeros_like(Waa)
        gradients['dWya'] = np.zeros_like(Wya)
        
        gradients['dba'] = np.zeros_like(ba)
        gradients['dby'] = np.zeros_like(by)
        gradients['da_next'] = np.zeros_like(a[0])

        # Lan truyền ngược qua thời gian
        for t in reversed(range(len(X))):
            dy = np.copy(y_pred[t])
            dy[Y[t]] -= 1
            gradients = self.rnn_step_backward(dy, gradients, x[t], a[t], a[t-1], parameters)
        
        return gradients, a[len(X)-1]


In [8]:
model = Simple_RNN(
    vocab_size,
    50
)

In [9]:
for idx in range(1, 30001):
    index = idx % len(examples)
    
    X = [None] + [char_to_ix[ch] for ch in examples[index]] 
    Y = X[1:] + [char_to_ix['\n']]
    loss = model.train(X, Y)

    if idx % 2000 == 0:
        print(f'loss: {loss} - iterator: {idx}')
        seed = 0

        for _ in range(7):
            
            indices = model.sample(seed)
            print(name_from_indices(indices, ix_to_char), end='')
            seed += 1
        
        print('\n')
        
    

loss: 25.5717881394549 - iterator: 2000
Miustolldoraurus
Jkecalosarapsaurusiolaydiskdctrldonasaurus
Kwtroligoraurus
Necalpsalantisaurus
Yuslepgoraurus
Daahosanassauristonaydiskdctrlepierosaurus
Usmciloraurus


loss: 9.618316101110358 - iterator: 4000
Livrosaurus
Gnfa
Gytrong
Ledalosaurus
Yusndonosaurus
Cabaselachus
Trraperneteris


loss: 37.79543806128584 - iterator: 6000
Ontosaurus
Lonba
Lusps
Onda
Ustonleroptisauor
Eebesphachorrilosaurus
Tosaurus


loss: 22.885498778584807 - iterator: 8000
Orusaurus
Midabersaurus
Mustrhoinicornos
Oncaltona
Xtrocherohurtos
Eeaertecartosaurus
Us


loss: 25.393736178808126 - iterator: 10000
Niutosaurus
Loia
Lustreodon
Nedalosaurus
Vrrocheoraurus
Daadosaurus
Ssokheriaurosaurus


loss: 21.85863548164 - iterator: 12000
Phvusaurus
Mjecanosaurus
Nyxtopharkethtosaurus
Pecalosaurus
Vrpthongmevbros
Ecalosaurus
Sus


loss: 19.54396573667707 - iterator: 14000
Oryuspeolophurus
Oracamptia
Ovutolomoravipitatisaurosaurus
Omabertia
Wurocikosaurus
Habcosaurus
Torapiora